# Qudit（多能级量子比特） 电路基础

*`tensorcircuit.quditcircuit.QuditCircuit` 的轻量入门*


## 概述

这个教程展示了如何使用 `tensorcircuit` 的 `QuditCircuit` API 来构建和模拟 **qudit（多能级量子比特）** 电路（d 级系统，其中 `d ≥ 3`）。

**要点**
- 创建一个 `QuditCircuit(nqudits, dim)` with dimension `dim ∈ [3, 36]`；
- 单比特门: `X`, `Z`, `H`, rotations `RX/RY/RZ` on selected levels `(j, k)`；
- 两比特门: `RXX`, `RZZ`, and the 广义受控求和 `CSUM` and 受控相位 `CPHASE`；
- 可得到波函数、概率、采样、期望值以及子系统投影；
- 样本和比特串使用36进制数字（`0–9A–Z`），其中`A = 10, ..., Z = 35`。



## 配置

In [1]:
import tensorcircuit as tc
from tensorcircuit.quditcircuit import QuditCircuit

tc.set_backend("numpy")  # or "jax", "tensorflow", "pytorch"
print("tensorcircuit version:", tc.__version__)

tensorcircuit version: 1.3.0


## 嗨，Qutrit！（维度为 13）

我们来准备一个**单独的 qudit* (`nqudits=1`, `dim=13`)， 对它应用一个广义的 Hadamard 门 `H`，让它进入一个等概率叠加态，然后观察最终的状态和概率。

In [2]:
c = QuditCircuit(nqudits=1, dim=13)
c.h(0)  # generalized Hadamard on the only qudit
psi = c.wavefunction()  # state vector of length 13^1 = 13
probs = c.probability()  # probability vector (length 3)
print(r"\psi:", psi)
print("P:", probs)

\psi: [0.2773501+0.j 0.2773501+0.j 0.2773501+0.j 0.2773501+0.j 0.2773501+0.j
 0.2773501+0.j 0.2773501+0.j 0.2773501+0.j 0.2773501+0.j 0.2773501+0.j
 0.2773501+0.j 0.2773501+0.j 0.2773501+0.j]
P: [0.07692308 0.07692308 0.07692308 0.07692308 0.07692308 0.07692308
 0.07692308 0.07692308 0.07692308 0.07692308 0.07692308 0.07692308
 0.07692308]


## 多量子比特基础

让我们来研究一下**两个三态量子比特**，使用 `H` 门和量子比特受控和门 `CSUM` 来创建一个最大纠缠态。

算符 `CSUM(control, target, cv=None)` 将控制位的数值加到目标位上，结果对 `dim` 取模。这是 CNOT 门的自然推广。如果传入 `cv`，则该门仅在控制位等于该值时激活（默认为 `None`）。

In [3]:
cq = QuditCircuit(nqudits=2, dim=3)  # two qutrits
cq.h(0)  # superpose control
cq.csum(0, 1)  # qudit CNOT analog (control=0, target=1)
psi = cq.wavefunction()
probs = cq.probability()
print(r"|\psi|^2 (length 3^2=9):", probs)

|\psi|^2 (length 3^2=9): [0.3333333 0.        0.        0.        0.3333333 0.        0.
 0.        0.3333333]


### 采样和 36 进制读数

采样会返回用 `0-9A-Z` 表示的、基于 `dim` 进制的字符串。例如，当 `dim=3` 时，字母表是 `0,1,2`：

In [4]:
samples = cq.sample(batch=512, format="count_dict_bin")  # e.g., '00', '11', '22'
samples

{'00': 175, '11': 165, '22': 172}

## 在选定能级上的单量子比特旋转

对于一个 qudit（多态量子位），旋转的目标是 `d` 个能级中的一个**双能级子空间**。

- `rx(index, theta, j=0, k=1)` 在能级 `j` 和 `k` 之间绕该嵌入的 SU(2) 的 X 轴旋转。
- `ry(index, theta, j=0, k=1)` 类似地，绕 Y 轴旋转。
- `rz(index, theta, j=0)` 对单个能级 `j` 应用 Z 相位。

> 提示：`(j, k)` 必须是 `[0, dim-1]` 中的不同整数。

In [5]:
import numpy as np

c = QuditCircuit(nqudits=1, dim=5)  # a ququint
c.h(0)  # start in equal superposition
c.rx(0, theta=np.pi / 3, j=1, k=3)  # rotate levels 1 and 3
c.rz(0, theta=np.pi / 5, j=4)  # add a phase to level 4
psi = c.wavefunction()
probs = c.probability()
psi, probs

(array([0.4472136 +0.j        , 0.38729832-0.2236068j ,
        0.4472136 +0.j        , 0.38729832-0.2236068j ,
        0.3618034 +0.26286554j], dtype=complex64),
 array([0.19999999, 0.19999997, 0.19999999, 0.19999997, 0.20000002],
       dtype=float32))

## 两个多能级量子比特的相互作用：`RXX`，`RZZ`

你可以对每个多能级量子比特的选定**子空间**进行操作来进行耦合：

- `rxx(q1, q2, theta, j1=0, k1=1, j2=0, k2=1)`
- `rzz(q1, q2, theta, j1=0, k1=1, j2=0, k2=1)`

这两个门都是量子比特 XX/ZZ 旋转的自然推广，但仅限于 `(j, k)` 子空间。

In [6]:
c2 = QuditCircuit(nqudits=2, dim=4)  # two ququarts
c2.h(0)
c2.h(1)
c2.rxx(0, 1, theta=np.pi / 4, j1=0, k1=2, j2=1, k2=3)
c2.rzz(0, 1, theta=np.pi / 7, j1=0, k1=1, j2=0, k2=1)
c2.probability()

array([0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625,
       0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625],
      dtype=float32)

## 局部算符的期望值

`expectation(*ops)` 计算一个或多个局部可观测量（算符）的期望值。每个可观测量是一个对 `(op, [site_indices])`，其中 `op` 是一个具有适当维度的张量（矩阵）。

In [7]:
# Example: build a diagonal operator on a single qutrit（三能级量子比特）（三能级量子比特） (dim=3)
import numpy as np

c = QuditCircuit(1, dim=3)
c.h(0)
op = np.diag([0.0, 0.5, 1.0])  # acts on subspace levels 0,1,2
expval = c.expectation((op, [0]))
expval

array(0.49999997+0.j, dtype=complex64)

### 应用任意门

直接使用 ``any`` API，输入对应的幺正矩阵即可。

In [8]:
d = 36
c = tc.QuditCircuit(2, dim=d)
h_matrix = tc.quditgates.h_matrix_func(d)
c.any(0, unitary=h_matrix)
csum_matrix = tc.quditgates.csum_matrix_func(d)
c.any(0, 1, unitary=csum_matrix)
c.sample(1024, format="count_dict_bin")

{'00': 35,
 '11': 28,
 '22': 28,
 '33': 25,
 '44': 33,
 '55': 30,
 '66': 34,
 '77': 25,
 '88': 22,
 '99': 22,
 'AA': 40,
 'BB': 38,
 'CC': 35,
 'DD': 17,
 'EE': 21,
 'FF': 27,
 'GG': 24,
 'HH': 26,
 'II': 27,
 'JJ': 24,
 'KK': 18,
 'LL': 27,
 'MM': 23,
 'NN': 38,
 'OO': 26,
 'PP': 31,
 'QQ': 27,
 'RR': 37,
 'SS': 28,
 'TT': 20,
 'UU': 34,
 'VV': 34,
 'WW': 32,
 'XX': 25,
 'YY': 29,
 'ZZ': 34}

## 注意事项和提示

- **维度**: `QuditCircuit` 会验证 `dim` 并且保持它在整个电路中的一致性。
- **波函数和概率**: `wavefunction()` 返回状态；`probability()` 返回一个长度为 `dim^n` 的向量。
- **采样**: `sample(batch, format="str")` 返回易于阅读的 36 进制字符串；使用 `format=None` 获取原始整数。
- **受控操作**: `csum(control, target, cv=None)` 泛化了 CNOT 门；`cv` 选择激活的控制值。
- **后端**: 根据需要通过 `tc.set_backend("numpy" | "jax" | "tensorflow" | "pytorch")` 切换后端。
- **互操作性**: 你仍然可以获取完整幺正矩阵的 `matrix()` 或类似 MPO 的 `quoperator()` 形式，用于高级工作流程。

所有这些函数都类似于 `tc.Circuit`。